# Additional NAICS Data Processing

**By: Calvin Chen and Irene Tang**

This file is for adding additionally features to the generated NAICS data from the `NAICS Code Data Generation` notebook. Feel free to add on here if there are other features you'd like to analyze in junction with previously generated NAICS Salesforce dataset.

In [1]:
import numpy as np
import pandas as pd
import re
import json
import time
import requests

---

## Add Lat/Long Information to Data

This script is to generate lat/long information to the collected Salesforce information via `ArcGIS`'s `batch_geocode` functionality.

### Credentials Helpers

In [2]:
def get_arcgis_access_token(client_id: str, client_secret: str) -> str:
    """
    Gets a new ArcGIS access token.
    """
    token = requests.get("https://www.arcgis.com/sharing/oauth2/token?client_id={}&grant_type=client_credentials&client_secret={}&f=pjson".format(client_id, client_secret))
    return token.json()['access_token']

def get_credentials(filename):
    """
    Getting credentials from given file.
    """
    with open(filename, 'r') as f:
        credentials = json.load(f)
        return credentials

def refresh_access_token(filename) -> None:
    """
    Writes the new passed in access token into the credentials file.
    """
    # Getting a new access token and writing it into `credentials.txt`
    credentials = get_credentials(filename)
    
    if filename == '../credentials/sf_credentials.txt':
        new_token = get_sf_access_token(credentials['client_id'], credentials['client_secret'], credentials['refresh_token'])
    elif filename == '../credentials/arcgis_credentials.txt':
        new_token = get_arcgis_access_token(credentials['client_id'], credentials['client_secret'])
    credentials['access_token'] = new_token
    
    # Writing new credentials back into `credentials.txt`
    with open(filename, 'r+') as f:
        json.dump(credentials, f, ensure_ascii=False)
    return credentials

### Data Extraction

In [10]:
# Read in generated data from previous notebook
all_data = pd.read_csv('../data/all_data.csv')

In [15]:
all_data.head().columns

Index(['Account__r.BillingStreet', 'Account__r.BillingPostalCode',
       'Account__r.BillingCity', 'Account__r.BillingState', 'Account__r.Name',
       'Award_Amount__c', 'Contract_Award_ID__c', 'DBE__c', 'MBE__c', 'WBE__c',
       'Opportunity__r.Id', 'Opportunity__r.Name',
       'Opportunity__r.Account.Name', 'Opportunity__r.Category__c',
       'Opportunity__r.Bid_Due__c', 'Opportunity__r.Bid_Post__c'],
      dtype='object')

In [16]:
all_data.isnull().sum()

Account__r.BillingStreet          0
Account__r.BillingPostalCode      0
Account__r.BillingCity            0
Account__r.BillingState          24
Account__r.Name                   0
Award_Amount__c                   0
Contract_Award_ID__c            538
DBE__c                            0
MBE__c                            0
WBE__c                            0
Opportunity__r.Id                 0
Opportunity__r.Name               0
Opportunity__r.Account.Name       0
Opportunity__r.Category__c        0
Opportunity__r.Bid_Due__c         0
Opportunity__r.Bid_Post__c        0
dtype: int64

In [4]:
def format_data(all_data: pd.DataFrame) -> list:
    """
    Formatting the data from the `all_data` dataframe into the correct format for the request payload.
    
    End payload to look like:
    [{'attributes': {'OBJECTID': 1,
       'Address': '25000 Avenue Stanford Suite 117',
       'City': 'Valencia',
       'Region': 'CA'}},
     {'attributes': {'OBJECTID': 2,
       'Address': '1 World Way',
       'City': 'Los Angeles',
       'Region': 'CA'}}]
    """
    # Making temp dataframe
    reduced_and_relabeled = (
        all_data[[
            'Account__r.BillingStreet', 
            'Account__r.BillingCity', 
            'Account__r.BillingState'
        ]]
        .rename(columns={
            'Account__r.BillingStreet': 'Address', 
            'Account__r.BillingCity': 'City', 
            'Account__r.BillingState': 'Region'
        })
        .drop_duplicates()
        # Getting an index column to link different objects before and after request is made
        .reset_index(drop=True)
        .reset_index()
        .rename(columns={
            'index': 'OBJECTID'
        })
    )
    
    # Finalizing dataframe attributes
    spatial_data = [
        {'attributes': val} 
            for val in list(
                reduced_and_relabeled
                # Only including certain characters in addresses (removing most special characters)
                .assign(Address=reduced_and_relabeled.Address.apply(lambda address: re.sub(r"[^a-zA-Z0-9. ]", "", address)))
                .T
                .to_dict()
                .values()
    )]

    return reduced_and_relabeled, spatial_data

In [5]:
def chunks(l, n):
    """
    Splits list into n-sized chunks and each chunk into the following format:
    
    {'records': [{'attributes': {'Address': '25000 Avenue Stanford Suite 117',
            'City': 'Valencia',
            'Region': 'CA'}},
        {'attributes': {'Address': '1 World Way',
            'City': 'Los Angeles',
            'Region': 'CA'}}]}
    """
    n = max(1, n)
    return [{'records': l[i:i+n]} for i in range(0, len(l), n)]

In [6]:
# Getting geospatial data
reduced_df, spatial_data = format_data(all_data)

# Geocoding addresses in batches of 100 (limit by URI length)
spatial_data_chunks = chunks(spatial_data, 100)

In [29]:
def generate_spatial_data(df_chunks, to_json=False):
    """
    Adds spatial data information to the `all_data` dataframe generated from earlier.
    """
    def check_error(data):
        """
        Check if the current data payload has an error.
        """
        # Check if the `access_token` worked. If not, refresh this call with a new access token.
        if 'error' in data:
            # Refresh the access token
            print("HTTP Request Access Token {} Error:".format(data['error']['code'], data['error']['message']))
            print("Trying to resolve issue by refreshing access token...")

            # Refreshing access token in credentials
            refresh_access_token('../credentials/arcgis_credentials.txt')
            return True
        
        return False
    
    # Getting credentials
    credentials = get_credentials('../credentials/arcgis_credentials.txt')
    
    # Making requests and appending locations to overall location list
    all_locations = []
    for chunk_idx in range(len(df_chunks)):
        start = time.time()
        print("Currently on chunk:", chunk_idx)
        res = requests.get("https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/geocodeAddresses?addresses={}&token={}&f=pjson".format(spatial_data_chunks[chunk_idx], credentials['access_token']))
        data = res.json()
        
        # Check if the data has an error
        if check_error(data):
            # Trying to make request again with new access token if there is an error
            return generate_spatial_data(df_chunks, to_json)
            
        all_locations.extend(data['locations'])
        finish = time.time()
        print("Request finished. Chunk took", finish - start, 'seconds.')
        
    # Exporting result object into JSON if desired.
    if to_json:
        full_json = json.dumps(all_locations)
        json_file = open('../data/arcgis_latlong_data.json', 'w')
        json_file.write(full_json)
        json_file.close()
        
    return all_locations

In [30]:
# Getting location data
locations = generate_spatial_data(spatial_data_chunks, to_json=True)

Currently on chunk: 0
Request finished. Chunk took 1.3169260025024414 seconds.
Currently on chunk: 1
Request finished. Chunk took 1.1272380352020264 seconds.
Currently on chunk: 2
Request finished. Chunk took 3.60090970993042 seconds.
Currently on chunk: 3
Request finished. Chunk took 1.5222229957580566 seconds.
Currently on chunk: 4
Request finished. Chunk took 1.5082166194915771 seconds.
Currently on chunk: 5
Request finished. Chunk took 1.4246199131011963 seconds.
Currently on chunk: 6
Request finished. Chunk took 1.6367056369781494 seconds.
Currently on chunk: 7
Request finished. Chunk took 1.2684261798858643 seconds.
Currently on chunk: 8
Request finished. Chunk took 1.122617244720459 seconds.
Currently on chunk: 9
Request finished. Chunk took 1.1019959449768066 seconds.
Currently on chunk: 10
Request finished. Chunk took 1.5155830383300781 seconds.
Currently on chunk: 11
Request finished. Chunk took 1.210702657699585 seconds.
Currently on chunk: 12
Request finished. Chunk took 1.

In [7]:
def add_loc_data(reduced_df, all_data, to_csv=False):
    """
    Adds location data generated by `generate_spatial_data` to the `all_data` dataframe in a new `final_df`.
    """
    # Load in the data
    file = open('../data/arcgis_latlong_data.json')
    data = json.load(file)
    
    # Creating a mapping between 'OBJECTID' and 'Latitude' and 'Longitude' coordinates.
    temp_mapping = {}
    for obj in data:
        object_id, status = obj['attributes']['ResultID'], obj['attributes']['Status']
        temp_mapping[object_id] = obj['location'] if status != 'U' else {'x': np.nan, 'y': np.nan}

    # Sorting the keys by ID so inserting into the dataframe will be easy.
    coords_mapping = dict(sorted(temp_mapping.items()))
    
    # Creating location mapping dataframe
    loc_mapping = reduced_df.copy(deep=True)
    loc_mapping['Latitude'] = [coord['y'] for coord in list(coords_mapping.values())]
    loc_mapping['Longitude'] = [coord['x'] for coord in list(coords_mapping.values())]
    loc_mapping['Full Address'] = loc_mapping['Address'] + ',' + loc_mapping['City'] + ',' + loc_mapping['Region']
    loc_mapping = loc_mapping[['Full Address', 'Latitude', 'Longitude']]
    
    # Creating `full_data` dataframe with `Full Address` w/o NaN values
    full_data = all_data.copy(deep=True)
    full_data['Full Address'] = full_data['Account__r.BillingStreet'] + ',' + full_data['Account__r.BillingCity'] + ',' + full_data['Account__r.BillingState']
    full_data = full_data[full_data['Full Address'].notnull()]
    
    # Merging data and removing NaN's in lat/long
    full_spatial_data = full_data.merge(loc_mapping, how='inner', left_on='Full Address', right_on='Full Address')
    final_df = full_spatial_data[full_spatial_data.Latitude.notnull() & full_spatial_data.Longitude.notnull()]
    
    # Adding country column (just U.S. for now)
    final_df['Country'] = 'United States'
    
    # Exporting dataframe to CSV if desired
    if to_csv:
        final_df.to_csv('../data/data_with_latlong.csv', index=False)
        
    return final_df

In [8]:
# Generating final dataframe with location data (lat, long) attached to each awarded opportunity.
final = add_loc_data(reduced_df, all_data, to_csv=True)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [9]:
final

,Account__r.BillingStreet,Account__r.BillingPostalCode,Account__r.BillingCity,Account__r.BillingState,Account__r.Name,Award_Amount__c,Contract_Award_ID__c,DBE__c,MBE__c,WBE__c,Opportunity__r.Id,Opportunity__r.Name,Opportunity__r.Account.Name,Opportunity__r.Category__c,Opportunity__r.Bid_Due__c,Opportunity__r.Bid_Post__c,Full Address,Latitude,Longitude,Country
0,25000 Avenue Stanford Suite 117,91355,Valencia,CA,Macktech Enterprises,10915.00,37569.0,False,False,False,0066g00003WK5EnAAL,Throw ToeBoards for Shot Put,General Services,Commodity,2021-04-08T18:00:00.000+0000,2021-04-05T22:50:59.000+0000,"25000 Avenue Stanford Suite 117,Valencia,CA",34.438209,-118.572685,United States
1,25000 Avenue Stanford Suite 117,91355,Valencia,CA,Macktech Enterprises,3297.80,32669.0,False,False,False,0066g00003WJ8vYAAT,"Ladder, Platform, 7-step",General Services,Commodity,2018-05-04T18:00:00.000+0000,2018-05-01T21:21:57.000+0000,"25000 Avenue Stanford Suite 117,Valencia,CA",34.438209,-118.572685,United States
2,25000 Avenue Stanford Suite 117,91355,Valencia,CA,Macktech Enterprises,1670.00,32375.0,False,False,False,0066g00003WJ8vrAAD,"Tool, ""SandVik Corormant""",General Services,Commodity,2018-02-27T19:00:00.000+0000,2018-02-21T20:47:03.000+0000,"25000 Avenue Stanford Suite 117,Valencia,CA",34.438209,-118.572685,United States
3,25000 Avenue Stanford Suite 117,91355,Valencia,CA,Macktech Enterprises,3177.00,31678.0,False,False,False,0066g00003WJ8u0AAD,"Bulk Salt, Delivered w Pneumatic Pump Truck to...",General Services,Commodity,2017-11-07T17:00:00.000+0000,2017-11-06T19:00:00.000+0000,"25000 Avenue Stanford Suite 117,Valencia,CA",34.438209,-118.572685,United States
4,25000 Avenue Stanford Suite 117,91355,Valencia,CA,Macktech Enterprises,11120.03,31850.0,False,False,False,0066g00003WJCpyAAH,"Generator, Dual Fuel Portable Inverter, LED Fl...",General Services,Commodity,2017-12-14T19:00:00.000+0000,2017-12-11T21:00:00.000+0000,"25000 Avenue Stanford Suite 117,Valencia,CA",34.438209,-118.572685,United States
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12327,40355 Delta Lane,93551,Palmdale,CA,Delta Scientific Corporation,820000.00,36191.0,False,False,False,0066g00003WJ9BVAA1,Venice Beach Ocean Front Walk Crash Ramps and ...,"Engineering Bureau, Public Works",Construction,2020-02-19T18:00:00.000+0000,2020-01-09T00:00:00.000+0000,"40355 Delta Lane,Palmdale,CA",34.615446,-118.133856,United States
12328,"1762 N. Neville St,",92865,Orange,CA,Drake Traffic Control Services,12000.00,36191.0,False,False,False,0066g00003WJ9BVAA1,Venice Beach Ocean Front Walk Crash Ramps and ...,"Engineering Bureau, Public Works",Construction,2020-02-19T18:00:00.000+0000,2020-01-09T00:00:00.000+0000,"1762 N. Neville St,,Orange,CA",33.816855,-117.857482,United States
12329,2025 E. Financial Way,91741,Glendora,CA,CalPortland Cement,50000.00,36191.0,False,False,False,0066g00003WJ9BVAA1,Venice Beach Ocean Front Walk Crash Ramps and ...,"Engineering Bureau, Public Works",Construction,2020-02-19T18:00:00.000+0000,2020-01-09T00:00:00.000+0000,"2025 E. Financial Way,Glendora,CA",34.129358,-117.826121,United States
12330,3000 N. 29th. Court,33020,Hollywood,FL,Epoxytec International Inc.,2958.16,36288.0,False,False,False,0066g00003WJ9ArAAL,BEIGE UROFLEX 4 GAL.KIT,General Services,Commodity,2020-04-28T18:00:00.000+0000,2020-04-23T20:06:22.000+0000,"3000 N. 29th. Court,Hollywood,FL",26.036666,-80.167298,United States


---

## **Adding a category column instead of separate `DBE`, `MBE`, and `WBE` columns (to overlay plots at the same time on Carto).**

In [8]:
# Read in data
arcgis_df = pd.read_csv('../data/data_with_latlong.csv')

In [5]:
# Join columns together
arcgis_df['Category'] = arcgis_df[['DBE__c', 'MBE__c', 'WBE__c']].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1
)

In [6]:
def find_category_name(booleans):
    """
    Find the category name for the list of booleans.
    """
    vals = list(map(lambda val: val == 'True', booleans.split(',')))
    if all(vals):
        return 'DBE, MBE, and WBE'
    elif vals[0] and vals[1]:
        return 'DBE and MBE'
    elif vals[1] and vals[2]:
        return 'MBE and WBE'
    elif vals[0] and vals[2]:
        return 'DBE and WBE'
    elif vals[0]:
        return 'DBE'
    elif vals[1]:
        return 'MBE'
    elif vals[2]:
        return 'WBE'
    elif (not vals[0] and not vals[1] and not vals[2]):
        return 'Not DBE, MBE, WBE'

In [7]:
# Find category name for each row
arcgis_df['Category'] = arcgis_df['Category'].apply(find_category_name)

In [8]:
# Export dataframe
arcgis_df.to_csv('../data/data_with_latlong_and_cat.csv', index=False)